### Universidad del Valle de Guatemala
### Deep Learning
### Laboratorio 7
### Andres Quezada 21085

In [1]:
import torch
import torch.nn as nn
import math

**Layer Normalization**

Esta clase implementa la normalización de capas, que es necesario en los transformadores para estabilizar el entrenamiento y mejorar la convergencia, se encarga de ajustar cada capa de activaciones al aprender los parámetros alpha y  bias. Estos parámetros se actualizan durante el entrenamiento para optimizar las activaciones de las capas. Por eso es que ambos son learnable parameters.


In [2]:
class LayerNormalization(nn.Module):
    def __init__(self, features: int, eps:float=10**-6) -> None:
        super().__init__()
        self.eps = eps # epsilon es un valor pequeño para evitar la división por cero
        self.alpha = nn.Parameter(torch.ones(features)) # alpha is a learnable parameter
        self.bias = nn.Parameter(torch.zeros(features)) # bias is a learnable parameter

    def forward(self, x):
        mean = x.mean(dim = -1, keepdim = True) # mean y std se calculan en la última dimensión
        std = x.std(dim = -1, keepdim = True)  # keepdim = True para mantener la dimensión de la media y la desviación está
        return self.alpha * (x - mean) / (std + self.eps) + self.bias
    # Regresa la normalización de x con alpha y bias

**Feed Forward Block**

Aquí se define la red feed-forward que se aplica después de la atención en cada capa del codificador y del decodificador. Es una red completamente conectada con dos capas lineales, una activación ReLU, y una capa de dropout para reducir el sobreajuste. Este bloque es importante para transformar las representaciones aprendidas por las capas de atención y generar las siguientes representaciones más complejas.

In [3]:
class FeedForwardBlock(nn.Module):
    def __init__(self, d_model: int, d_ff: int, dropout: float) -> None:
        super().__init__()
        self.linear_1 = nn.Linear(d_model, d_ff) # primera capa lineal con d_model entradas y d_ff salidas
        self.dropout = nn.Dropout(dropout) # dropout para regularización
        self.linear_2 = nn.Linear(d_ff, d_model)  # segunda capa lineal con d_ff entradas y d_model salidas

    def forward(self, x):
        # regresa la salida de la segunda capa lineal después de aplicar dropout y relu a la primera capa lineal
        return self.linear_2(self.dropout(torch.relu(self.linear_1(x))))

**Input Embeddings**

En esta clase se conviertem los tokens de entrada en las representaciones vectoriales (embeddings) que se utilizarán para procesar la secuencia de entrada. Este paso es necesario en los modelos de lenguaje, ya que transforman palabras o tokens discretos en vectores que permiten capturar relaciones semánticas entre ellos. Permitiendo la búsqueda de atención en oraciones y textos largos.

In [4]:
class InputEmbeddings(nn.Module):
    def __init__(self, d_model: int, vocab_size: int) -> None:
        super().__init__()
        self.d_model = d_model # tamaño del embedding
        self.vocab_size = vocab_size # tamaño del vocabulario
        self.embedding = nn.Embedding(vocab_size, d_model) # embedding de tamaño vocab_size x d_model

    def forward(self, x):
        return self.embedding(x) * math.sqrt(self.d_model)
    # regresa el embedding de x multiplicado por la raíz cuadrada de d_model
    

**Positional Encoding**

Aquí se implementa el Positional Encoding, que es necesario en la arquitectura de transformadores ya que estos modelos no tienen una estructura secuencial como las redes recurrentes u otras que hemos visto en clase. Entonces, el transformador no sabe cuál es el orden de las palabras de la secuencia, y para lograr que sepa, hay que introducir información sobre el orden, por eso se asigna una representación única a cada posición de la secuencia.

Se utilizan funciones seno y coseno para generar patrones periódicos que varían según la posición de la palabra en la secuencia, permitiendo que el modelo distinga el orden relativo entre las palabras.

In [5]:
class PositionalEncoding(nn.Module):

    def __init__(self, d_model: int, seq_len: int, dropout: float) -> None:
        super().__init__()
        self.d_model = d_model # tamaño del embedding
        self.seq_len = seq_len # longitud de la secuencia
        self.dropout = nn.Dropout(dropout) # dropout para regularización
        
        pe = torch.zeros(seq_len, d_model) # inicializa pe con ceros
        position = torch.arange(0, seq_len, dtype=torch.float).unsqueeze(1) # posición de la secuencia
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model)) # término de división
        pe[:, 0::2] = torch.sin(position * div_term) # seno en las posiciones pares
        pe[:, 1::2] = torch.cos(position * div_term)  # coseno en las posiciones impares
        pe = pe.unsqueeze(0)  # (1, seq_len, d_model)
        self.register_buffer('pe', pe) # registra pe como buffer

    def forward(self, x):
        x = x + (self.pe[:, :x.shape[1], :]).requires_grad_(False) # (batch, seq_len, d_model)
        return self.dropout(x)
    # regresa x más el positional encoding

**Residual Connection**

Aquí se implementan las conexiones que permiten que las entradas originales se sumen a las salidas de las capas posteriores. Esto ayuda a evitar el problema del vanishing gradient, facilitando el flujo de gradientes durante el entrenamiento y mejorando la estabilidad de las redes profundas. El dropout y la normalización de capas también se aplican para evitar el sobreajuste y mantener la estabilidad durante el entrenamiento.

In [6]:
class ResidualConnection(nn.Module):
    
    def __init__(self, features: int, dropout: float) -> None:
        super().__init__()
        # Aplica dropout para evitar el sobreajuste
        self.dropout = nn.Dropout(dropout)
        # Normalización de capas para estabilizar el entrenamiento
        self.norm = LayerNormalization(features)
    
    def forward(self, x, sublayer):
        # La salida de la subcapa normalizada se suma a la entrada original 
        return x + self.dropout(sublayer(self.norm(x)))


**Multi Head Attention Block**

Aquí se implementa el multi-head attention, que permite que el modelo aprenda diferentes relaciones entre los elementos de la secuencia simultáneamente a través de diferentes puntos de atención. Cada uno de estos puntos proyecta los queries, keys y values en subespacios diferentes, y luego los combina para formar una representación final enriquecida con toda la información.

El mecanismo de atención permite que el modelo preste atención a diferentes partes de la secuencia de entrada, ponderando la importancia de cada palabra en función de otras. Mejorando la capacidad del modelo para capturar múltiples tipos de relaciones entre las palabras o tokens.

In [7]:
class MultiHeadAttentionBlock(nn.Module):

    def __init__(self, d_model: int, h: int, dropout: float) -> None:
        super().__init__()
        self.d_model = d_model
        self.h = h
        # Verifica que el número de dimensiones (d_model) sea divisible por el número de cabezas (h)
        assert d_model % h == 0, "d_model is not divisible by h"
        
        # Tamaño de las proyecciones por cabeza
        self.d_k = d_model // h  
        # Definición de las capas lineales para proyecciones de queries, keys, values y la salida
        self.w_q = nn.Linear(d_model, d_model, bias=False)
        self.w_k = nn.Linear(d_model, d_model, bias=False)
        self.w_v = nn.Linear(d_model, d_model, bias=False)
        self.w_o = nn.Linear(d_model, d_model, bias=False)
        self.dropout = nn.Dropout(dropout)

    @staticmethod
    def attention(query, key, value, mask, dropout: nn.Dropout):
        # Calcula las puntuaciones de atención escaladas
        d_k = query.shape[-1]
        attention_scores = (query @ key.transpose(-2, -1)) / math.sqrt(d_k)
        
        # Aplica la máscara, si existe, para ignorar ciertas posiciones
        if mask is not None:
            attention_scores.masked_fill_(mask == 0, -1e9)
        
        # Se aplica softmax para normalizar las puntuaciones en probabilidades
        attention_scores = attention_scores.softmax(dim=-1)
        
        # Aplica dropout para evitar el sobreajuste en las puntuaciones de atención
        if dropout is not None:
            attention_scores = dropout(attention_scores)
        
        # Calcula la salida de la atención multiplicando por los valores (values)
        return (attention_scores @ value), attention_scores

    def forward(self, q, k, v, mask):
        # Aplica las proyecciones lineales para obtener los queries, keys y values
        query = self.w_q(q)
        key = self.w_k(k)
        value = self.w_v(v)

        # Reorganiza los tensores para aplicar la atención múltiple (multi-head attention)
        query = query.view(query.shape[0], query.shape[1], self.h, self.d_k).transpose(1, 2)
        key = key.view(key.shape[0], key.shape[1], self.h, self.d_k).transpose(1, 2)
        value = value.view(value.shape[0], value.shape[1], self.h, self.d_k).transpose(1, 2)

        # Calcula la atención con múltiples cabezas
        x, self.attention_scores = MultiHeadAttentionBlock.attention(query, key, value, mask, self.dropout)

        # Reorganiza las salidas y aplica la proyección final
        x = x.transpose(1, 2).contiguous().view(x.shape[0], -1, self.h * self.d_k)
        
        # Proyección final de la salida combinada de las cabezas de atención
        return self.w_o(x)


**Encoder Block**

Aquí empieza la fase codificadora del transformador, se aplica atención múltiple a la entrada, seguido de un bloque feed-forward. Cada operación dentro del bloque (la self-attention y el feed-forward) se acompaña de una residual connection, lo que ayuda a preservar la información de las capas anteriores, evitando la degradación de la señal. También se aplica normalización de capas y dropout para estabilizar y regularizar el entrenamiento.

El EncoderBlock es un bloque clave en la arquitectura del transformador, ya que permite procesar una secuencia de entrada para generar representaciones útiles que capturan tanto dependencias locales como globales dentro de la secuencia.


In [8]:
class EncoderBlock(nn.Module):

    def __init__(self, features: int, self_attention_block: MultiHeadAttentionBlock, feed_forward_block: FeedForwardBlock, dropout: float) -> None:
        super().__init__()
        # Bloque de autoatención múltiple
        self.self_attention_block = self_attention_block
        # Bloque feed-forward que sigue a la atención
        self.feed_forward_block = feed_forward_block
        # Se crean dos conexiones residuales: una para la atención y otra para el feed-forward
        self.residual_connections = nn.ModuleList([ResidualConnection(features, dropout) for _ in range(2)])

    def forward(self, x, src_mask):
        # Aplica la primera conexión residual con el bloque de autoatención
        x = self.residual_connections[0](x, lambda x: self.self_attention_block(x, x, x, src_mask))
        # Aplica la segunda conexión residual con el bloque feed-forward
        x = self.residual_connections[1](x, self.feed_forward_block)
        return x


**Encoder**

Esta clase representa el conjunto completo de capas del codificador en un transformador. Un transformador codificador está formado por múltiples capas, donde cada una de estas capas aplica una combinación de autoatención y bloques feed-forward con conexiones residuales. En esta clase, las capas se procesan de manera secuencial, y después de pasar por todas ellas, se aplica una normalización de capa final para estabilizar las representaciones generadas.

In [9]:
class Encoder(nn.Module):

    def __init__(self, features: int, layers: nn.ModuleList) -> None:
        super().__init__()
        # Las capas del codificador, normalmente una lista de EncoderBlocks
        self.layers = layers
        # Normalización de capa al final de todas las capas del codificador
        self.norm = LayerNormalization(features)

    def forward(self, x, mask):
        # Pasa la entrada a través de cada capa del codificador secuencialmente
        for layer in self.layers:
            x = layer(x, mask)
        # Aplica normalización de capa al resultado final después de todas las capas
        return self.norm(x)


**Decoder Block**

Y el complemeto del bloque anterior, está aqui en el bloque del decodificador en el transformador. El decodificador es responsable de generar las salidas basadas tanto en las representaciones del codificador como en las secuencias de salida generadas hasta el momento. 

Este bloque realiza tres operaciones principales:

1. Autoatención sobre la secuencia de salida generada hasta el momento.

2. Atención cruzada sobre las salidas del codificador, permitiendo que el decodificador se enfoque en diferentes partes de la secuencia de entrada.

3. Bloque feed-forward para realizar una transformación adicional en las representaciones.

Cada una de estas operaciones está envuelta en una conexión residual, lo que ayuda a mantener la estabilidad del entrenamiento y evita la pérdida de información entre las capas.


In [10]:
class DecoderBlock(nn.Module):

    def __init__(self, features: int, self_attention_block: MultiHeadAttentionBlock, cross_attention_block: MultiHeadAttentionBlock, feed_forward_block: FeedForwardBlock, dropout: float) -> None:
        super().__init__()
        # Bloque de autoatención para la secuencia de salida generada
        self.self_attention_block = self_attention_block
        # Bloque de atención cruzada, que toma en cuenta las representaciones del codificador
        self.cross_attention_block = cross_attention_block
        # Bloque feed-forward para transformar las representaciones
        self.feed_forward_block = feed_forward_block
        # Tres conexiones residuales: una para la autoatención, una para la atención cruzada, y una para el feed-forward
        self.residual_connections = nn.ModuleList([ResidualConnection(features, dropout) for _ in range(3)])

    def forward(self, x, encoder_output, src_mask, tgt_mask):
        # Paso 1
        x = self.residual_connections[0](x, lambda x: self.self_attention_block(x, x, x, tgt_mask))
        # Paso 2
        x = self.residual_connections[1](x, lambda x: self.cross_attention_block(x, encoder_output, encoder_output, src_mask))
        # Paso 3
        x = self.residual_connections[2](x, self.feed_forward_block)
        return x


**Decoder**

Esta representa el decodificador completo, que toma las salidas del codificador y genera la secuencia de salida objetivo, aplicando la serie de capas que incluyen autoatención y atención cruzada. Al igual que el codificador, el decodificador está compuesto por múltiples capas, y cada una de estas capas utiliza conexiones residuales para mantener la información de entrada intacta. Finalmente, se aplica una normalización de capa a la salida del decodificador.


In [11]:
class Decoder(nn.Module):

    def __init__(self, features: int, layers: nn.ModuleList) -> None:
        super().__init__()
        # Las capas del decodificador, una lista de DecoderBlocks
        self.layers = layers
        # Normalización de capa al final del decodificador
        self.norm = LayerNormalization(features)

    def forward(self, x, encoder_output, src_mask, tgt_mask):
        # Pasa la entrada (secuencia generada hasta el momento) a través de cada capa del decodificador
        for layer in self.layers:
            x = layer(x, encoder_output, src_mask, tgt_mask)
        # Aplica normalización de capa al resultado final después de todas las capas
        return self.norm(x)


**Projection Layer**

Este es casi que el paso final, aqui se implementa una capa de proyección final en el transformador. Su propósito es proyectar las representaciones de salida generadas por el decodificador de la dimensión que traen al tamaño del vocabulario. Esta capa es necesaria en tareas de modelado de lenguaje, ya que convierte las representaciones del modelo en logits sobre el vocabulario, que posteriormente se pueden convertir en probabilidades para predecir la palabra siguiente en una secuencia.

In [12]:
class ProjectionLayer(nn.Module):

    def __init__(self, d_model, vocab_size) -> None:
        super().__init__()
        # Capa lineal para proyectar de d_model al tamaño del vocabulario
        self.proj = nn.Linear(d_model, vocab_size)

    def forward(self, x) -> None:
        # Proyección final: (batch, seq_len, d_model) --> (batch, seq_len, vocab_size)
        return self.proj(x)



**Transformer** (Optimus Prime)

Finalmente, esta clase es la implementación completa del modelo de transformador, que incluye tanto el codificador como el decodificador. 

Este modelo se utiliza principalmente en tareas de secuencia a secuencia, como la traducción automática, donde la secuencia de entrada se codifica en una representación interna, y luego esa representación se usa para generar la secuencia de salida.

El transformador tiene tres funciones principales:

- encode: Convierte la secuencia de entrada en una representación interna utilizando el codificador.
- decode: Genera la secuencia de salida basada en las representaciones del codificador y la secuencia objetivo.
- project: Proyecta las representaciones finales en el espacio del vocabulario para obtener logits sobre las palabras del vocabulario.

In [13]:
class Transformer(nn.Module):

    def __init__(self, encoder: Encoder, decoder: Decoder, src_embed: InputEmbeddings, tgt_embed: InputEmbeddings, src_pos: PositionalEncoding, tgt_pos: PositionalEncoding, projection_layer: ProjectionLayer) -> None:
        super().__init__()
        # Codificador
        self.encoder = encoder
        # Decodificador
        self.decoder = decoder
        # Embeddings para la secuencia de input
        self.src_embed = src_embed
        # Embeddings para la secuencia objetivo
        self.tgt_embed = tgt_embed
        # Codificación posicional para la secuencia de input
        self.src_pos = src_pos
        # Codificación posicional para la secuencia de output
        self.tgt_pos = tgt_pos
        # Capa de proyección para transformar las representaciones a logits sobre el vocabulario
        self.projection_layer = projection_layer

    def encode(self, src, src_mask):
        # Embedding y codificación posicional para la secuencia de inputt
        src = self.src_embed(src)
        src = self.src_pos(src)
        # Pasa la secuencia embebida a través del codificador
        return self.encoder(src, src_mask)
    
    def decode(self, encoder_output: torch.Tensor, src_mask: torch.Tensor, tgt: torch.Tensor, tgt_mask: torch.Tensor):
        # Embedding y codificación posicional para la secuencia de salida
        tgt = self.tgt_embed(tgt)
        tgt = self.tgt_pos(tgt)
        # Pasa la secuencia embebida y la salida del codificador al decodificador
        return self.decoder(tgt, encoder_output, src_mask, tgt_mask)
    
    def project(self, x):
        # Proyecta las representaciones en logits sobre el vocabulario
        # (batch, seq_len, vocab_size)
        return self.projection_layer(x)


**build_transformer**

Aunque esta no es una de las clases, igual la quiero explicar por que es parte del proceso.

Esta función construye un modelo de transformador completo, configurado para procesar secuencias de entrada y generar secuencias de salida. Tiene como parámetros el tamaño del vocabulario, las longitudes de las secuencias, el tamaño de los embeddings (d_model), el número de capas (N), el número de cabezas de atención (h), el tamaño de la red feed-forward (d_ff), y la tasa de dropout.

La función define tanto el codificador como el decodificador con sus respectivos bloques de autoatención y atención cruzada, y finalmente los une en un modelo completo de transformador. Al final, también se inicializan los parámetros del transformador utilizando la inicialización de Xavier, que ayuda a mejorar la convergencia del modelo durante el entrenamiento.




In [14]:
def build_transformer(src_vocab_size: int, tgt_vocab_size: int, src_seq_len: int, tgt_seq_len: int, d_model: int=512, N: int=6, h: int=8, dropout: float=0.1, d_ff: int=2048) -> Transformer:
    # Embeddings de la secuencia de entrada y salida
    src_embed = InputEmbeddings(d_model, src_vocab_size)
    tgt_embed = InputEmbeddings(d_model, tgt_vocab_size)

    # Codificaciones posicionales para las secuencias de entrada y salida
    src_pos = PositionalEncoding(d_model, src_seq_len, dropout)
    tgt_pos = PositionalEncoding(d_model, tgt_seq_len, dropout)
    
    # Construcción de los bloques del codificador
    encoder_blocks = []
    for _ in range(N):
        # Bloque de autoatención múltiple para el codificador
        encoder_self_attention_block = MultiHeadAttentionBlock(d_model, h, dropout)
        # Bloque feed-forward
        feed_forward_block = FeedForwardBlock(d_model, d_ff, dropout)
        # Bloque del codificador
        encoder_block = EncoderBlock(d_model, encoder_self_attention_block, feed_forward_block, dropout)
        encoder_blocks.append(encoder_block)

    # Construcción de los bloques del decodificador
    decoder_blocks = []
    for _ in range(N):
        # Bloque de autoatención múltiple para la secuencia de salida
        decoder_self_attention_block = MultiHeadAttentionBlock(d_model, h, dropout)
        # Bloque de atención cruzada con las salidas del codificador
        decoder_cross_attention_block = MultiHeadAttentionBlock(d_model, h, dropout)
        # Bloque feed-forward
        feed_forward_block = FeedForwardBlock(d_model, d_ff, dropout)
        # Bloque del decodificador
        decoder_block = DecoderBlock(d_model, decoder_self_attention_block, decoder_cross_attention_block, feed_forward_block, dropout)
        decoder_blocks.append(decoder_block)
    
    # Ensamblar el codificador y decodificador completos
    encoder = Encoder(d_model, nn.ModuleList(encoder_blocks))
    decoder = Decoder(d_model, nn.ModuleList(decoder_blocks))
    
    # Capa de proyección para convertir las representaciones en logits sobre el vocabulario
    projection_layer = ProjectionLayer(d_model, tgt_vocab_size)
    
    # Construir el modelo de transformador completo
    transformer = Transformer(encoder, decoder, src_embed, tgt_embed, src_pos, tgt_pos, projection_layer)

    # Inicialización de los parámetros del modelo usando Xavier uniform para mejorar la convergencia
    for p in transformer.parameters():
        if p.dim() > 1:
            nn.init.xavier_uniform_(p)
    
    return transformer


**Referencias**

Vaswani, A. (2017). Attention is all you need. Advances in Neural Information Processing Systems. Recuperado de: https://arxiv.org/pdf/1706.03762

PyTorch. (2024) Torch.nn Documentation. Recuperado de: https://pytorch.org/docs/stable/nn.html

